In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ2: In which phase of the technology life cycle path are the technologies?
# - Interest in topic: #Papers on a topic over time 
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
#- Publikationstyp: Workshop -> Conference -> Journal 

In [3]:
data = pd.read_csv('./data/data.csv',usecols = ['Publication Year','Article Citation Count','Author Keywords','Publisher'], dtype={'Author Keywords': str, 'Publisher': str})
data.head()

,Publication Year,Author Keywords,Article Citation Count,Publisher
0,2009,software engineering;knowledge management;lear...,28.0,IEEE
1,2009,NaN,NaN,IEEE
2,2011,software engineering;computer and information ...,11.0,IEEE
3,2010,human aspects of software engineering;software...,3.0,IEEE
4,2012,software engineering theory;theory;explanation...,17.0,IEEE


In [4]:
data.columns

Index(['Publication Year', 'Author Keywords', 'Article Citation Count',
       'Publisher'],
      dtype='object')

In [5]:
def IntrestInTopic(year):
    databyyear = data[data['Publication Year'] == year]
    

In [6]:
def Count(column,count):
    cnt = Counter()
    for item in data[column]: 
        
        cnt[item] += 1
    return cnt.most_common(count)

#_similarity = cosine_similarity()

In [7]:
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time

#df = data[['Authors','Cites']].groupby('Authors').sum()


#d = df['Cites'].loc[df['Cites'] > 400]
#d.plot.bar()
#print(df['Cites'])



result = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='Article Citation Count',
                        aggfunc=np.sum,
                        fill_value = 0)
#result

In [8]:
#- Publikationstyp: Workshop -> Conference -> Journal 
#data['Author Keywords'] = data['Author Keywords'].str.split(';',expand=False)

#data['Author Keywords'].head()

#data[data['Author Keywords'].str.split(';',expand=False).Contains]
#Mit isin Liste abfragen
#Erst Topic Filtern und dann weiter Arbeiten

test = {}

words = Counter()
#data[data['Publication Year'] == year]
for year in range(2009,2021,1):
    for columns in data[data['Publication Year'] == year].itertuples():

        text = columns[2] #
        if type(text) is str:
            for topic in text.split(';'):
                words[topic] += 1

        test[year] = list(words)

    print(words.most_common(40))   

[('software engineering', 43), ('Model checking', 9), ('Reliability', 7), ('software testing', 6), ('software architecture', 6), ('Software Engineering', 6), ('Markov processes', 5), ('software development', 5), ('software design', 5), ('software process improvement', 4), ('debugging', 4), ('Validation', 4), ('Software and System Safety', 4), ('Formal methods', 4), ('Stochastic processes', 4), ('Test design', 4), ('open source', 4), ('agile software development', 4), ('Modeling techniques', 4), ('Queuing theory', 4), ('software management', 3), ('metrics', 3), ('domain-specific architectures', 3), ('Software science', 3), ('Requirements/Specifications', 3), ('Testing and Debugging', 3), ('maintainability', 3), ('software metrics', 3), ('Management', 3), ('availability', 3), ('and serviceability', 3), ('extreme programming', 3), ('Design notations and documentation', 3), ('State diagrams', 3), ('software/program verification', 3), ('spreadsheets', 2), ('Modeling', 2), ('analysis', 2), (

In [9]:
print(test[2019])

['software engineering', 'knowledge management', 'learning software organization', 'software process improvement', 'systematic review', 'spreadsheets', 'type checking', 'debugging', 'spatial analysis', 'Classification', 'software testing', 'test design techniques', 'testing techniques', 'unit testing techniques.', 'curriculum', 'Software Engineering Body of Knowledge', 'SWEBOK', 'computer science education', 'Software engineering education', 'game-based learning', 'software management', 'software simulation', 'professional ethics', 'codes of ethics', 'codes of good practice', 'professional licensing', 'software engineering practice', 'Modeling', 'analysis', 'diagrams', 'communication', 'visualization', 'visual syntax', 'concrete syntax.', 'Software engineering life cycle', 'software quality', 'software project success', 'software process', 'multiple-case study.', 'General', 'Life cycle', 'Software Quality/SQA', 'Multiple-case study', 'metrics', 'control', 'project management', 'budget 

In [10]:
import spacy
from spacy.symbols import NOUN


nlp = spacy.load("en_core_web_sm")

cnt = Counter()
tokenlist = []

for columns in data.itertuples():

    text = columns[2] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
    if type(text) is str:
        doc = nlp(text)
        
        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
            token_text = token.text
            token_pos = token.pos_
            token_dep = token.dep_
            token_ent = token.ent_type_
            
            if token.pos == NOUN:
                if not token.is_stop:
                    print(f"{token_text} | {token_ent} | {token.tag_}")
                    cnt[token_text] += 1
                    tokenlist.append([token_text,columns[2],token_ent])


software |  | NN
management;learning |  | NN
software |  | NN
process |  | NN
review |  | NN
software |  | NN
engineering;computer |  | NN
information |  | NN
science |  | NN
architecture;core |  | NN
body |  | NN
knowledge;SWEBOK |  | NNS
aspects |  | NNS
software |  | NN
engineering;software |  | NN
engineering |  | NN
education;computer |  | NN
science |  | NN
education;software |  | NN
projects;software |  | NN
development |  | NN
software |  | NN
engineers |  | NNS
software |  | NN
engineering |  | NN
analysis |  | NN
software |  | NN
engineering |  | NN
innovation;diffusion |  | NN
software |  | NN
software |  | NN
innovations |  | NNS
software |  | NN
software |  | NN
development |  | NN
software |  | NN
literature |  | NN
review |  | NN
Classification;software |  | NN
design |  | NN
techniques;testing | PERSON | NN
testing |  | NN
techniques |  | NNS
software |  | NN
science |  | NN
education |  | NN
Software |  | NN
engineering |  | NN
education;game | ORG | NN
simulation |  |